In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from core import ROOT_PATH
from core.embedding_models import EmbeddingModel, FastTextEmbeddingModel
from core.models.clustering_model import ClusteringModel

tqdm.pandas()

In [2]:
data_path = Path("data/data_vacancies_processed_1k.csv")
df = pd.read_csv(ROOT_PATH / data_path, index_col=0)
df["work_skills"] = df.work_skills.apply(eval)
df.head()

,id,custom_position,schedule,salary_from,salary_to,offer_education_id,education_name,education_is_base,education_order_num,city_id,list_regions,work_skills,tags_id
0,48202096,Сварщик-сборщик,полный рабочий день,60000,120000,0,любое,True,0,2,[4],"[сварочные работы, сборка изделий по чертежам,...",NaN
1,48202097,Сварщик-монтажник,полный рабочий день,60000,120000,0,любое,True,0,2,[4],"[монтажные работы, строительные работы, электр...",NaN
2,48202098,Слесарь-сборщик,полный рабочий день,60000,80000,0,любое,True,0,2,[4],"[работа на фрезерных станках, слесарный ремонт...",NaN
3,48202356,Грузчик-упаковщик,частичная занятость,30000,35000,0,любое,True,0,1,[3],"[комплектация товара, маркировка, стрессоустой...","[6, 9]"
4,48202357,Грузчик-упаковщик,частичная занятость,30000,35000,0,любое,True,0,57,"[181, 182, 183, 185, 186, 187, 188, 189, 190, ...","[маркировка, стрессоустойчивость, погрузочно-р...","[6, 9]"


In [3]:
fasttext_model = FastTextEmbeddingModel()
hf_model = EmbeddingModel()

In [4]:
unique_skills = df.work_skills.explode().unique()
unique_skills_embeddings_fasttext = {
    k: fasttext_model.generate(k) for k in tqdm(unique_skills)
}
# unique_skills_embeddings_hf = {k: hf_model.generate(k) for k in tqdm(unique_skills)}

  0%|          | 0/896 [00:00<?, ?it/s]

In [5]:
embeddings = (
    df.work_skills.apply(lambda x: list(map(unique_skills_embeddings_fasttext.get, x)))
    .apply(lambda x: np.mean(x, axis=0))
    .tolist()
)
embeddings = np.array(embeddings)

In [6]:
model = ClusteringModel()
model = model.train(embeddings)

/Users/Konstantin.Grotov/Library/Caches/pypoetry/virtualenvs/rabota-ru-fYDRv1HV-py3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [7]:
model.predict(embeddings[0])

13

In [8]:
model_path = ROOT_PATH / "checkpoints/clustering_model_fasttext.pkl"
model.save_model(model_path)

Model saved at /Users/Konstantin.Grotov/Documents/programming/projects/jbr/salary-prediction/checkpoints/clustering_model_fasttext.pkl


In [9]:
model = model.load_model(model_path)

Model successfully loaded


In [10]:
model.predict(embeddings[0])

13